# Decision tree math for citizen science classifications

##### Coleman Krawczyk (University of Portsmouth); converted to iPython notebook by Kyle Willett (University of Minnesota)

Extends analysis of *SpaceWarps* project to full decision trees consisting of multiple choice questions (eg, *Galaxy Zoo*)

### Case 1: One question in the workflow

Let $M$ be a confusion matrix of the form:

$M_{ij} = P[i_{vote}|j_{ans}]$

where $P$ is the probability that a user votes $i$ when the true answer is $j$. A singular matrix (where elements along the diagonal $i=j$ are equal to 1 and all other $M_{i\ne j}=0$) would correspond to a perfect classifier. 

In [1]:
# Example classifiers for SpaceWarps

import numpy as np

pc = np.identity(2)
rc = np.zeros_like(pc) + 0.5

print "Perfect classifier"
print pc
print "\nRandom classifier"
print rc

Perfect classifier
[[ 1.  0.]
 [ 0.  1.]]

Random classifier
[[ 0.5  0.5]
 [ 0.5  0.5]]


Using Bayes' theorem, we can solve for the probability that $j$ will be the answer if the **last** vote given was $i$:

$P[j_{ans}|i_{vote}] = \frac{P[i_{vote}|j_{ans}]P[j_{ans}]}{P[i_{vote}]}$

where $P[j_{ans}]$ is the prior probability that $j$ is the answer.

The first term in the numerator is our definition of the confusion matrix $M$ above. The term in the denominator is the normalization. We can put this in terms of $M$ by noting that the total probabilities must sum to 1 (assuming an answer must exist), so:

$1 = \sum\limits_{j_{ans}}P[j_{ans}|i_{vote}]$.

Multiplying both sides by $P[i_{vote}]$:

$P[i_{vote}] = P[i_{vote}]\sum\limits_{j_{ans}}P[j_{ans}|i_{vote}]$.

From Bayes' theorem:

$P[j_{ans}|i_{vote}] = \frac{P[i_{vote}|j_{ans}]P[j_{ans}]}{P[i_{vote}]} = \frac{M_{ij}P[j_{ans}]}{P[i_{vote}]}$

and so

$P[i_{vote}] = P[i_{vote}]\sum\limits_{j_{ans}}\frac{M_{ij}P[j_{ans}]}{P[i_{vote}]} = \sum\limits_{j_{ans}}M_{ij}P[j_{ans}]$.

This makes sense - the probability of a user voting for $i$ is the sum of all the weighted probabilities that can lead there given the true answer $j$.

Substituting both expressions in the numerator and denominator, this gives:

$P[j_{ans}|i_{vote}] = \frac{M_{ij}P[j_{ans}]}{\sum\limits_{j_{ans}} \left(M_{ij}P[j_{ans}]\right)}$

### Case 2: Multiple questions in the workflow

Let the probability that the *previous* question leads to the current question $k$ be:

$P[T_{k-1} \rightarrow T_k] = P[T_k]$

Then the probability that the question is missed is simply the complement of $P[T_k]$:

$P[T_{k-1} \not\rightarrow T_k] = P[\bar T_k] = 1 - P[T_k]$

The confusion matrix $M_{ij}$ now needs to take into account the probability that the question is either seen or missed. If the question is seen, then:

$M_{i\ne miss,j} = P[T_k]P[i_{vote}|j_{ans}]$.

If the question is missed:

$M_{i=miss,j} = P[\bar T_k)P[miss_{vote}|j_{ans}] = \left(1 - P[T_k]\right)P[miss_{vote}|j_{ans}]$.

In this case, the confusion matrix requires a new column to indicate the probability of voting for each choice with the true answer is "miss". A prior is also required for the chance of a "true miss". 

To normalize the total probability, the columns of $M$ must sum to 1 for all $j_{ans}$:

$\sum\limits_{i\ne miss}\left(P[T_k]P[i_{vote}|j_{ans}]\right) + P[\bar T_k]P[miss_{vote}|j_{ans}] = 1$.

We assume that the probability that the previous question leads to this does not affect the likelihood of it being a miss, so $P[T_k]$ can be removed from the sum:

$P[T_k]\sum\limits_{i\ne miss}\left(P[i_{vote}|j_{ans}]\right) + P[\bar T_k]P[miss_{vote}|j_{ans}] = 1$

Now if the task is answered by the user, they can't vote for "miss" by definition. So the summation of the first term above must be 1:

$P[T_k] + P[\bar T_k]P[miss_{vote}|j_{ans}] = 1$

Substituting in the definition of $P[\bar T_k]$:

$P[T_k] + \left(1 - P[T_k]\right)P[miss_{vote}|j_{ans}] = 1$

$\implies P[miss_{vote}|j_{ans}] = 1$

So this gives two separate scenarios: one where the question is seen and answered, and one where it's missed due to an earlier branch in the tree. We can invert both scenarios to get the posterior version using Bayes' Theorem in the same way that we did above. 

#### If question is answered:

$P[j_{ans}|i_{vote}] = \frac{M_{ij}P[j_{ans}]}{\sum\limits_{j_{ans}}\left(M_{ij}P[j_{ans}]\right)}$

Substituting:

$P[j_{ans}|i_{vote}] = \frac{P[T_k] P[i_{vote}|j_{ans}] P[j_{ans}]}{P[T_k]\sum\limits_{j_{ans}}\left( P[i_{vote}|j_{ans}]P[j_{ans}]\right)} = \frac{P[i_{vote}|j_{ans}] P[j_{ans}]}{\sum\limits_{j_{ans}}\left( P[i_{vote}|j_{ans}]P[j_{ans}]\right)}$

So if the question is answered, the ***probability that the task was seen cancels out*** - none of the remaining terms depend on $P[T_k]$. The confusion matrix $M_{ij}$ can be calculated without this information. 

*What if the question isn't answered?*

#### If question is not answered (a miss):

$P[j_{ans}|miss_{vote}] = \frac{M_{i=miss,j}P[j_{ans}]}{P[miss_{vote}]}$

Variables on RHS:

$M_{i=miss,j} = P[\bar T_k] P[miss_{vote}|j_{ans}] = P[\bar T_k]$ (*from above*)

$P[miss_{vote}] = P[\bar T_k] \sum\limits_{j_{ans}} P[j_{ans}]$

Substituting:

$P[j_{ans}|miss_{vote}] = \frac{P[\bar T_k]P[j_{ans}]}{P[\bar T_k] \sum\limits_{j_{ans}} P[j_{ans}]}$

The probabilities over all $j_{ans}$ must sum to 1, so $P[\bar T_k]$ cancels out:

$P[j_{ans}|miss_{vote}] = \frac{P[\bar T_k] P[j_{ans}]}{P[\bar T_k]} = P[j_{ans}]$


So if the question is **not** answered, then the posterior probability is exactly the same as the prior. No information is provided about a given task from the classifications where the question is not seen!

### Conclusions

The behavior of a user can be characterized by a probability matrix $M_{ij}$ for some set of answers to each task. If a question is not seen by a given user, that information is independent of the information provided for the "true" answer to the question being characterized by $M$.  